# CalgaryHacks2026 IBM Workshop - Agentic Repo Analyzer

Welcome! In this notebook, you will:
- configure a local repo analysis workflow
- run multiple role-based agents to critique a project
- generate a practical execution checklist for the hackathon window


## Before you run

1. Install Python (version 3.12.12 used for development)
2. Install UV and run `uv sync`
3. Generate your Ollama API Key
4. Create a `.env` file in this project folder using `cp .env.example .env`
5. Add your API key to the `.env` file like this:

```env
OLLAMA_API_KEY=your_real_api_key_here
```
6. Select the correct kernel (top-right in Jupyter)


## How to run

1. Update `ANALYZE_REPO_PATH` to the local repo you want to analyze.
2. Set `END_DATETIME` and `START_DATETIME` for your hackathon window.
3. Run all cells from top to bottom.


## Expected outputs

- `ScoutAgent`: summary of what exists in the repo
- `CriticAgent`: risks and weaknesses
- `PlannerAgent`: prioritized 24-hour plan
- `ReviewerAgent`: pass/fail + revision notes
- `JudgingAgent`: 1-10 judge-style scoring
- `Final Report`: consolidated summary
- `demo_output.md`: generated task checklist


In [1]:
# Import dependencies
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
from ollama import Client
from IPython.display import Markdown, display


In [ ]:
# Set workshop configuration
START_DATETIME = "2026-02-11T09:00"
END_DATETIME = "2026-02-12T09:00"
USE_CURRENT_TIME_AS_START = True

ANALYZE_REPO_PATH = Path.home() / "Documents" / "Projects" / "calgary-accessibility"
OUTPUT_REPO_PATH = Path.cwd() / "notebooks"
TASKS_FILENAME = "demo_output.md"

HACKATHON_OBJECTIVE = "Build a website where people can find local doctors who are accepting new patients and can speak the user's language."

MODEL_NAME = "qwen3-next:80b"
OLLAMA_HOST = "https://ollama.com"
MAX_FILES = 30
MAX_AGENT_STEPS = 8
ALLOWED_EXTS = {".py", ".ts", ".tsx", ".js", ".jsx", ".md", ".yaml", ".yml"}
MAX_CHARS_PER_FILE = 2000


In [3]:
# Load API key and create client
load_dotenv()

api_key = os.getenv("OLLAMA_API_KEY", "").strip()
if not api_key or api_key == "your_real_api_key_here":
    raise RuntimeError("Missing OLLAMA_API_KEY in .env")

client = Client(
    host=OLLAMA_HOST,
    headers={"Authorization": f"Bearer {api_key}"},
)

print(f"Connected to {OLLAMA_HOST}")


Connected to https://ollama.com


In [4]:
# Build repository snapshot
if USE_CURRENT_TIME_AS_START:
    now = datetime.now()
    end = datetime.fromisoformat(END_DATETIME)
    hours_left = max(0.0, round((end - now).total_seconds() / 3600, 2))
else:
    start = datetime.fromisoformat(START_DATETIME)
    end = datetime.fromisoformat(END_DATETIME)
    hours_left = round((end - start).total_seconds() / 3600, 2)

sampled_files = []
for root, _, names in os.walk(ANALYZE_REPO_PATH):
    for name in names:
        rel_path = str((Path(root) / name).relative_to(ANALYZE_REPO_PATH))
        if Path(rel_path).suffix.lower() not in ALLOWED_EXTS:
            continue

        file_path = ANALYZE_REPO_PATH / rel_path
        try:
            content = file_path.read_text(encoding="utf-8", errors="ignore")
        except OSError:
            continue

        sampled_files.append({"path": rel_path, "content": content[:MAX_CHARS_PER_FILE]})
        if len(sampled_files) >= MAX_FILES:
            break
    if len(sampled_files) >= MAX_FILES:
        break

snapshot = {
    "repo_path": str(ANALYZE_REPO_PATH),
    "sampled_files": sampled_files,
}

sampled_text = ""
for entry in snapshot["sampled_files"]:
    sampled_text += f"\n### FILE: {entry['path']}\n{entry['content']}\n"

shared_context = (
    f"Objective:\n{HACKATHON_OBJECTIVE}\n\n"
    f"Repo path: {snapshot['repo_path']}\n"
    f"Sampled files: {len(snapshot['sampled_files'])}\n\n"
    f"Sampled content:\n{sampled_text}"
)

print("Analyzing repo:", ANALYZE_REPO_PATH)
print("Objective:", HACKATHON_OBJECTIVE)
print("Hours left:", hours_left)
print("Files sampled:", len(snapshot["sampled_files"]))


Analyzing repo: C:\Users\sdsaj\Documents\Projects\calgary-accessibility
Objective: Build a website where people can find local doctors who are accepting new patients and can speak the user's language.
Hours left: 11.07
Files sampled: 12


In [5]:
# Define agent prompts
scout_prompt = """
You are ScoutAgent. Analyze this repository context and return strict JSON with keys:
existing_features (array of strings), architecture_summary (string),
gaps (array of strings), confidence (high|medium|low).
"""

critic_prompt = """
You are CriticAgent. Given repository context and Scout output, return strict JSON with keys:
weaknesses (array of {issue,severity,impact}), technical_risks (array of strings),
demo_risks (array of strings).
"""

planner_prompt = """
You are PlannerAgent for a student 24-hour hackathon.
Use Scout and Critic outputs to create a realistic plan.
Return strict JSON with keys:
top_features_to_build (array of strings),
not_now_features (array of strings),
phased_plan (array of {phase,time_block,deliverables}),
pitch_angle (string), judging_checklist (array of strings).
"""

synth_prompt = """
You are the CoordinatorAgent. Convert the JSON inputs into concise markdown for students.
Format sections exactly as: Overview, Existing Strengths, Weaknesses,
Top Features to Build, 24-Hour Execution Plan, Demo Pitch, First 3 Tasks.
Keep under 500 words.
"""

reviewer_prompt = """
You are ReviewerAgent. Check if the report is realistic for the hours available,
aligns with the objective, and avoids over-scoping.
Return strict JSON with keys:
pass (boolean), issues (array of strings), revision_instructions (string).
"""

judge_prompt = """
You are JudgingAgent for a hackathon.
Score the current project state on a 1-10 scale (10 is best) and return strict JSON with keys:
- topic_alignment (number)
- innovation (number)
- solution_effectiveness (number)
- technical_challenge (number)
- ui_design_effort (number)
- overall_score (number)
- strengths (array of strings)
- weaknesses (array of strings)
- judge_summary (string)
Keep scores realistic for an in-progress student project.
"""


In [6]:
# Connection + model availability check
try:
    model_data = client.list()
except Exception as exc:
    raise ConnectionError(
        "Could not connect to Ollama API. Check internet access and API key."
    ) from exc

available_models = [m.get("model", "") for m in model_data.get("models", [])]

if not available_models:
    raise RuntimeError("Connected, but no models are available for this account.")

if MODEL_NAME in available_models:
    selected_model = MODEL_NAME
    display(Markdown(f"Found requested model: `{selected_model}`"))
else:
    selected_model = available_models[0]
    display(Markdown(f"`{MODEL_NAME}` not found. Using `{selected_model}` instead."))

n = 10
preview = ", ".join(available_models[:n])
display(Markdown(f"Available models (first n): `{preview}`"))

Found requested model: `qwen3-next:80b`

Available models (first n): `cogito-2.1:671b, glm-4.6, glm-4.7, glm-5, kimi-k2:1t, kimi-k2.5, kimi-k2-thinking, qwen3-coder:480b, qwen3-next:80b, qwen3-coder-next`

In [7]:
# Run agent workflow
state = {
    "objective": HACKATHON_OBJECTIVE,
    "hours": hours_left,
    "scout": None,
    "critic": None,
    "planner": None,
    "report": None,
    "review": None,
    "judging": None,
    "issues": [],
    "history": [],
}

progress_lines = []

progress_lines.append("Running: ScoutAgent")
print(progress_lines[-1])
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": scout_prompt},
        {"role": "user", "content": shared_context},
    ],
)
raw = response.get("message", {}).get("content", "")
start = raw.find("{")
end = raw.rfind("}")
if start != -1 and end != -1 and end > start:
    try:
        state["scout"] = json.loads(raw[start : end + 1])
    except json.JSONDecodeError:
        state["scout"] = {"raw": raw}
else:
    state["scout"] = {"raw": raw}
state["history"].append({"action": "scout"})

critic_input = (
    f"{shared_context}\n\nScout output:\n{json.dumps(state['scout'], indent=2)}"
)
progress_lines.append("Running: CriticAgent")
print(progress_lines[-1])
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": critic_prompt},
        {"role": "user", "content": critic_input},
    ],
)
raw = response.get("message", {}).get("content", "")
start = raw.find("{")
end = raw.rfind("}")
if start != -1 and end != -1 and end > start:
    try:
        state["critic"] = json.loads(raw[start : end + 1])
    except json.JSONDecodeError:
        state["critic"] = {"raw": raw}
else:
    state["critic"] = {"raw": raw}
state["history"].append({"action": "critic"})

planner_input = (
    f"Objective:\n{HACKATHON_OBJECTIVE}\n"
    f"Available hours: {hours_left}\n"
    f"Scout output:\n{json.dumps(state['scout'], indent=2)}\n\n"
    f"Critic output:\n{json.dumps(state['critic'], indent=2)}"
)
progress_lines.append("Running: PlannerAgent")
print(progress_lines[-1])
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": planner_prompt},
        {"role": "user", "content": planner_input},
    ],
)
raw = response.get("message", {}).get("content", "")
start = raw.find("{")
end = raw.rfind("}")
if start != -1 and end != -1 and end > start:
    try:
        state["planner"] = json.loads(raw[start : end + 1])
    except json.JSONDecodeError:
        state["planner"] = {"raw": raw}
else:
    state["planner"] = {"raw": raw}
state["history"].append({"action": "plan"})

MAX_REVISE_STEPS = max(1, MAX_AGENT_STEPS - 3)
loop_steps = []
for revise_step in range(1, MAX_REVISE_STEPS + 1):
    prior_review = json.dumps(state["review"], indent=2) if state["review"] else "none"
    synth_input = (
        f"Objective:\n{HACKATHON_OBJECTIVE}\n"
        f"Available hours: {hours_left}\n"
        f"Scout:\n{json.dumps(state['scout'], indent=2)}\n\n"
        f"Critic:\n{json.dumps(state['critic'], indent=2)}\n\n"
        f"Planner:\n{json.dumps(state['planner'], indent=2)}\n\n"
        f"Prior review (if any):\n{prior_review}"
    )

    progress_lines.append(
        f"Running: CoordinatorAgent (revise step {revise_step}/{MAX_REVISE_STEPS})"
    )
    print(progress_lines[-1])
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": synth_prompt},
            {"role": "user", "content": synth_input},
        ],
    )
    draft = response.get("message", {}).get("content", "").strip()
    if draft:
        state["report"] = draft

    if not state.get("report"):
        state["issues"].append("CoordinatorAgent returned an empty report draft.")
        loop_steps.append(f"Step {revise_step}/{MAX_REVISE_STEPS}: no draft returned")
        continue

    reviewer_input = (
        f"Objective:\n{HACKATHON_OBJECTIVE}\n"
        f"Available hours: {hours_left}\n"
        f"Report draft:\n{state['report']}"
    )
    progress_lines.append(
        f"Running: ReviewerAgent (revise step {revise_step}/{MAX_REVISE_STEPS})"
    )
    print(progress_lines[-1])
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": reviewer_prompt},
            {"role": "user", "content": reviewer_input},
        ],
    )
    review_raw = response.get("message", {}).get("content", "")
    start = review_raw.find("{")
    end = review_raw.rfind("}")
    if start != -1 and end != -1 and end > start:
        try:
            state["review"] = json.loads(review_raw[start : end + 1])
        except json.JSONDecodeError:
            state["review"] = {
                "pass": False,
                "issues": ["ReviewerAgent returned non-JSON output."],
                "revision_instructions": "Tighten scope and keep the plan realistic for available hours.",
            }
    else:
        state["review"] = {
            "pass": False,
            "issues": ["ReviewerAgent returned non-JSON output."],
            "revision_instructions": "Tighten scope and keep the plan realistic for available hours.",
        }

    review_ready = bool(state["review"].get("pass", False))
    state["history"].append(
        {"action": "revise", "iteration": revise_step, "review_ready": review_ready}
    )
    loop_steps.append(
        f"Step {revise_step}/{MAX_REVISE_STEPS}: {'ready' if review_ready else 'needs_tuning'}"
    )
    if review_ready:
        loop_steps.append("Stopped early: report marked ready by ReviewerAgent.")
        break

if not state.get("report"):
    state["issues"].append("No report generated; creating one final fallback draft.")
    fallback_input = (
        f"Objective:\n{HACKATHON_OBJECTIVE}\n"
        f"Available hours: {hours_left}\n"
        f"Scout:\n{json.dumps(state['scout'], indent=2)}\n\n"
        f"Critic:\n{json.dumps(state['critic'], indent=2)}\n\n"
        f"Planner:\n{json.dumps(state['planner'], indent=2)}"
    )
    progress_lines.append("Running: CoordinatorAgent (fallback draft)")
    print(progress_lines[-1])
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": synth_prompt},
            {"role": "user", "content": fallback_input},
        ],
    )
    state["report"] = response.get("message", {}).get("content", "").strip()

if not bool((state.get("review") or {}).get("pass", False)):
    loop_steps.append(
        "Completed max revise steps; using best draft plus reviewer suggestions."
    )

judge_input = (
    f"Objective:\n{HACKATHON_OBJECTIVE}\n"
    f"Available hours: {hours_left}\n"
    f"Scout:\n{json.dumps(state.get('scout') or {}, indent=2)}\n\n"
    f"Critic:\n{json.dumps(state.get('critic') or {}, indent=2)}\n\n"
    f"Planner:\n{json.dumps(state.get('planner') or {}, indent=2)}\n\n"
    f"Current report:\n{state.get('report') or ''}"
)
progress_lines.append("Running: JudgingAgent")
print(progress_lines[-1])
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": judge_prompt},
        {"role": "user", "content": judge_input},
    ],
)
judge_raw = response.get("message", {}).get("content", "")
start = judge_raw.find("{")
end = judge_raw.rfind("}")
if start != -1 and end != -1 and end > start:
    try:
        state["judging"] = json.loads(judge_raw[start : end + 1])
    except json.JSONDecodeError:
        state["judging"] = {
            "topic_alignment": 0,
            "innovation": 0,
            "solution_effectiveness": 0,
            "technical_challenge": 0,
            "ui_design_effort": 0,
            "overall_score": 0,
            "strengths": ["JudgingAgent returned non-JSON output."],
            "weaknesses": ["Could not score the project."],
            "judge_summary": "Judging output unavailable.",
        }
else:
    state["judging"] = {
        "topic_alignment": 0,
        "innovation": 0,
        "solution_effectiveness": 0,
        "technical_challenge": 0,
        "ui_design_effort": 0,
        "overall_score": 0,
        "strengths": ["JudgingAgent returned non-JSON output."],
        "weaknesses": ["Could not score the project."],
        "judge_summary": "Judging output unavailable.",
    }

scout_json = state.get("scout") or {}
critic_json = state.get("critic") or {}
planner_json = state.get("planner") or {}
review_json = state.get("review") or {
    "pass": False,
    "issues": ["No review output."],
    "revision_instructions": "",
}
judging_json = state.get("judging") or {}

student_report = state.get("report") or ""
judging_lines = [
    "## Judging Summary",
    f"- Overall Score: {judging_json.get('overall_score', 'N/A')}/10",
    f"- Summary: {judging_json.get('judge_summary', 'No judging summary available.')}",
    "",
    "### Judge Strengths",
]
strengths = judging_json.get("strengths", [])
if isinstance(strengths, list) and strengths:
    judging_lines.extend([f"- {item}" for item in strengths])
else:
    judging_lines.append("- No strengths provided.")

judging_lines.extend(["", "### Judge Weaknesses"])
weaknesses = judging_json.get("weaknesses", [])
if isinstance(weaknesses, list) and weaknesses:
    judging_lines.extend([f"- {item}" for item in weaknesses])
else:
    judging_lines.append("- No weaknesses provided.")

final_report = student_report.strip()
if final_report:
    final_report += "\n\n" + "\n".join(judging_lines)
else:
    final_report = "\n".join(judging_lines)

progress_markdown = ["### Agent Execution Trace"]
progress_markdown.extend([f"- {line}" for line in progress_lines])

workflow_lines = ["### Workflow Loop Steps"]
if loop_steps:
    workflow_lines.extend([f"- {step}" for step in loop_steps])
else:
    workflow_lines.append("- No revise steps were recorded.")

action_lines = [
    "### Agent Action Summary",
    f"- ScoutAgent: mapped repo context ({len(scout_json.get('existing_features', [])) if isinstance(scout_json, dict) else 0} features, {len(scout_json.get('gaps', [])) if isinstance(scout_json, dict) else 0} gaps).",
    f"- CriticAgent: flagged risks ({len(critic_json.get('weaknesses', [])) if isinstance(critic_json, dict) else 0} weaknesses, {len(critic_json.get('technical_risks', [])) if isinstance(critic_json, dict) else 0} technical risks, {len(critic_json.get('demo_risks', [])) if isinstance(critic_json, dict) else 0} demo risks).",
    f"- PlannerAgent: proposed execution plan ({len(planner_json.get('top_features_to_build', [])) if isinstance(planner_json, dict) else 0} top features, {len(planner_json.get('phased_plan', [])) if isinstance(planner_json, dict) else 0} phases).",
    f"- ReviewerAgent: validation status = {'pass' if review_json.get('pass', False) else 'needs_tuning'} ({len(review_json.get('issues', [])) if isinstance(review_json, dict) else 0} issues).",
    f"- JudgingAgent: scored project at {judging_json.get('overall_score', 'N/A')}/10 and produced a judging rationale.",
]
if state["issues"]:
    action_lines.append(f"- Controller: recorded {len(state['issues'])} runtime issue(s).")

display(Markdown("\n".join(progress_markdown + [""] + workflow_lines + [""] + action_lines)))
display(Markdown("### Final Report"))
display(Markdown(final_report))


### Agent Run Plan
- ScoutAgent: scans sampled repo files and maps existing features/gaps.
- CriticAgent: identifies technical and demo risks.
- PlannerAgent: proposes a prioritized build plan for the remaining hours.
- ReviewerAgent: checks scope realism and requests revisions if needed.
- JudgingAgent: scores current project quality and provides judging rationale.

Running: ScoutAgent
Running: CriticAgent
Running: PlannerAgent
Running: CoordinatorAgent (revise step 1/5)
Running: ReviewerAgent (revise step 1/5)
Running: CoordinatorAgent (revise step 2/5)
Running: ReviewerAgent (revise step 2/5)
Running: CoordinatorAgent (revise step 3/5)
Running: ReviewerAgent (revise step 3/5)
Running: CoordinatorAgent (revise step 4/5)
Running: ReviewerAgent (revise step 4/5)
Running: JudgingAgent


### Agent Execution Trace
- Running: ScoutAgent
- Running: CriticAgent
- Running: PlannerAgent
- Running: CoordinatorAgent (revise step 1/5)
- Running: ReviewerAgent (revise step 1/5)
- Running: CoordinatorAgent (revise step 2/5)
- Running: ReviewerAgent (revise step 2/5)
- Running: CoordinatorAgent (revise step 3/5)
- Running: ReviewerAgent (revise step 3/5)
- Running: CoordinatorAgent (revise step 4/5)
- Running: ReviewerAgent (revise step 4/5)
- Running: JudgingAgent

### Workflow Loop Steps
- Step 1/5: needs_tuning
- Step 2/5: needs_tuning
- Step 3/5: needs_tuning
- Step 4/5: ready
- Stopped early: report marked ready by ReviewerAgent.

### Agent Action Summary
- ScoutAgent: mapped repo context (7 features, 9 gaps).
- CriticAgent: flagged risks (9 weaknesses, 5 technical risks, 5 demo risks).
- PlannerAgent: proposed execution plan (5 top features, 4 phases).
- ReviewerAgent: validation status = pass (0 issues).
- JudgingAgent: scored project at 3/10 and produced a judging rationale.

### Final Report

**Overview**  
Build a website for finding local doctors accepting new patients who speak the user's language. Current system has basic CRUD and language filtering but lacks critical features for core functionality.  

**Existing Strengths**  
REST API CRUD, language filtering endpoint, doctor detail view, React Router, MongoDB storage, client-side language/gender filtering.  

**Weaknesses**  
- No "accepting new patients" filtering (high severity)  
- Incomplete PATCH route for acceptance status  
- Location data unused in queries  
- No admin auth (anyone can delete records)  
- No multi-criteria filtering (language + location + status)  
- No location validation or server-side filtering  

**Top Features to Build**  
Admin authentication, server-side multi-criteria filtering, location search with validation, fixed PATCH route, responsive search form.  

**11-Hour Execution Plan**  
- **Backend Fixes (3h)**: Admin auth (JWT/roles), fix PATCH, server location filtering + validation.  
- **Frontend (3h)**: Search form with location/status/language inputs, API calls, remove client-side filtering.  
- **Data & Test (2h)**: Seed sample data, test filters, validate location inputs.  
- **Polish & Demo (3h)**: Responsive UI, demo script, final bug fixes.  

**Demo Pitch**  
"Empowering patients to find nearby doctors who accept new patients and speak their language with real-time multi-criteria filtering—no more guessing, just clear local healthcare options."  

**First 3 Tasks**  
1. Implement admin authentication (JWT/role checks).  
2. Fix PATCH route to update `accepting_patients` status.  
3. Add server-side location filtering with input validation.

## Judging Summary
- Overall Score: 3/10
- Summary: Project has a solid technical foundation but fails to implement core functionality required for its objective. Critical gaps in acceptance status filtering, location-based search, and authentication prevent the solution from being functional. While the architecture is sound, the current state lacks essential features needed for a viable healthcare search tool, making it a high-risk demo with significant work remaining.

### Judge Strengths
- REST API CRUD operations implemented
- Language-based filtering endpoint functional
- MongoDB data storage with Mongoose schema
- React Router for navigation
- Basic Bootstrap UI structure

### Judge Weaknesses
- No 'accepting new patients' status filtering
- Incomplete PATCH route for acceptance status updates
- Location data unused in queries
- No authentication for admin operations
- Client-side filtering instead of server-side processing
- No multi-criteria filtering support
- No location data validation
- High risk of demo failure due to missing core features

In [8]:
# Build and write task checklist
lines = [
    "# Hackathon Tasks",
    "",
    f"Objective: {HACKATHON_OBJECTIVE}",
    f"Time Budget (hours): {hours_left}",
    "",
    "## Top Features",
]

top_features = planner_json.get("top_features_to_build", [])
if isinstance(top_features, list) and top_features:
    for item in top_features:
        lines.append(f"- [ ] {item}")
else:
    lines.append("- [ ] Define top features from planner output")

lines.extend(["", "## 24-Hour Plan"])
phased_plan = planner_json.get("phased_plan", [])
if isinstance(phased_plan, list) and phased_plan:
    for phase in phased_plan:
        if not isinstance(phase, dict):
            continue
        phase_name = phase.get("phase", "Phase")
        block = phase.get("time_block", "TBD")
        deliverables = phase.get("deliverables", [])
        lines.append(f"- [ ] {phase_name} ({block})")
        if isinstance(deliverables, list):
            for item in deliverables[:5]:
                lines.append(f"  - {item}")
else:
    lines.append("- [ ] Build phased plan from planner output")

lines.extend(["", "## Judge Snapshot (1-10)"])
lines.append(f"- Topic Alignment: {judging_json.get('topic_alignment', 'N/A')}")
lines.append(f"- Innovation: {judging_json.get('innovation', 'N/A')}")
lines.append(f"- Solution Effectiveness: {judging_json.get('solution_effectiveness', 'N/A')}")
lines.append(f"- Technical Challenge: {judging_json.get('technical_challenge', 'N/A')}")
lines.append(f"- UI/Design Effort: {judging_json.get('ui_design_effort', 'N/A')}")
lines.append(f"- Overall Score: {judging_json.get('overall_score', 'N/A')}")

lines.extend(["", "## Judging Summary"])
lines.append(f"- {judging_json.get('judge_summary', 'No judging summary available.')}")

lines.extend(["", "## Defer / Not Now"])
not_now = planner_json.get("not_now_features", [])
if isinstance(not_now, list) and not_now:
    for item in not_now:
        lines.append(f"- [ ] {item}")
else:
    lines.append("- [ ] Capture out-of-scope ideas")

lines.extend(["", "## Review Notes"])
issues = review_json.get("issues", [])
if isinstance(issues, list) and issues:
    for issue in issues:
        lines.append(f"- {issue}")
else:
    lines.append("- No major issues flagged by ReviewerAgent.")

lines.append("")
tasks_md = "\n".join(lines)
out_path = OUTPUT_REPO_PATH / TASKS_FILENAME
out_path.write_text(tasks_md, encoding="utf-8")
print("Wrote:", out_path)


Wrote: c:\Users\sdsaj\Documents\Projects\calgaryhacks2026-ibm-workshop\demo_output.md


## Try this next
- Point `ANALYZE_REPO_PATH` at a different repository and compare outputs.
- Tighten the `HACKATHON_OBJECTIVE` to test how plans change under constraints.
- Swap the model and review differences in score confidence and task quality.
